In [33]:
import pandas as pd
import numpy as np

import os
import random

from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib

random.seed(123)
np.random.seed(123)

pd.set_option('display.max_columns', 200)

root_dir = '/'.join(os.path.realpath(__name__).split('/')[:-2])
input_dir = root_dir + '/input_data'
persist_dir = root_dir + '/persist'

def save_check_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print('>> {} << directory created.'.format(dir_path.split('/')[-1]))
        
    else:
        print(dir_path)
        print('>> {} << directory already exists.'.format(dir_path.split('/')[-1]))

save_check_dir(persist_dir)

/Users/300029144/Documents/kaggle/home_credit_default/home-credit-default/persist
>> persist << directory already exists.


In [34]:
application_path = input_dir + '/application_train.csv'
bureau_path = input_dir + '/bureau.csv'
bureau_balance_path = input_dir + '/bureau_balance.csv'

In [35]:
application = pd.read_csv(application_path)

In [4]:
bureau = pd.read_csv(bureau_path)
bureau_bal = pd.read_csv(bureau_balance_path)

In [7]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [16]:
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


# Data Cleaning

In [ ]:
bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
bureau['DAYS_CREDIT_UPDATE'][bureau['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT'] < -40000] = np.nan

# HandCrafted Features

In [28]:
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

In [29]:
groupby = bureau.groupby('SK_ID_CURR')
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})

g = groupby['DAYS_CREDIT'].agg('count').reset_index()
g.rename(index=str, columns={'DAYS_CREDIT': 'bureau_number_of_past_loans'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['CREDIT_TYPE'].agg('nunique').reset_index()
g.rename(index=str, columns={'CREDIT_TYPE': 'bureau_number_of_loan_types'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['bureau_credit_active_binary'].agg('mean').reset_index()
g.rename(index=str, columns={'bureau_credit_active_binary': 'bureau_credit_active_binary'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['AMT_CREDIT_SUM_DEBT'].agg('sum').reset_index()
g.rename(index=str, columns={'AMT_CREDIT_SUM_DEBT': 'bureau_total_customer_debt'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['AMT_CREDIT_SUM'].agg('sum').reset_index()
g.rename(index=str, columns={'AMT_CREDIT_SUM': 'bureau_total_customer_credit'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['AMT_CREDIT_SUM_OVERDUE'].agg('sum').reset_index()
g.rename(index=str, columns={'AMT_CREDIT_SUM_OVERDUE': 'bureau_total_customer_overdue'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['CNT_CREDIT_PROLONG'].agg('sum').reset_index()
g.rename(index=str, columns={'CNT_CREDIT_PROLONG': 'bureau_average_creditdays_prolonged'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

g = groupby['bureau_credit_enddate_binary'].agg('mean').reset_index()
g.rename(index=str, columns={'bureau_credit_enddate_binary': 'bureau_credit_enddate_percentage'}, inplace=True)
features = features.merge(g, on=['SK_ID_CURR'], how='left', suffixes=(None, None))

features['bureau_average_of_past_loans_per_type'] = \
    features['bureau_number_of_past_loans'] / features['bureau_number_of_loan_types']

features['bureau_debt_credit_ratio'] = \
    features['bureau_total_customer_debt'] / features['bureau_total_customer_credit']

features['bureau_overdue_debt_ratio'] = \
    features['bureau_total_customer_overdue'] / features['bureau_total_customer_debt']

In [31]:
BUREAU_AGGREGATION_RECIPIES = [('CREDIT_TYPE', 'count'),
                               ('CREDIT_ACTIVE', 'size')
                               ]
for agg in ['mean', 'min', 'max', 'sum', 'var']:
    for select in [
                   'AMT_CREDIT_SUM',
                   'AMT_CREDIT_SUM_DEBT',
                   'AMT_CREDIT_SUM_LIMIT',
                   'AMT_CREDIT_SUM_OVERDUE',
                   'AMT_CREDIT_MAX_OVERDUE',
                   'CNT_CREDIT_PROLONG',
                   'CREDIT_DAY_OVERDUE',
                   'DAYS_CREDIT',
                   'DAYS_CREDIT_ENDDATE',
                   'DAYS_CREDIT_UPDATE'
                   ]:
        BUREAU_AGGREGATION_RECIPIES.append((select, agg))
BUREAU_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], BUREAU_AGGREGATION_RECIPIES)]

In [41]:
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(BUREAU_AGGREGATION_RECIPIES):
    group_object = bureau.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        features = features.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

In [49]:
persist_path = persist_dir + '/bureau.pkl'
joblib.dump(features, persist_path)

['/Users/300029144/Documents/kaggle/home_credit_default/home-credit-default/persist/bureau.pkl']